In [1]:
from __future__ import annotations
import functools
import itertools
import re

from typing import Union


## Utils

In [2]:
# custom types
Char = str
Dataset = list[str]

In [3]:
input_dir = 'input/'
    
def input_for(day: int) -> Dataset:
    try:
        with(open(f'input/day-{day}.txt', 'r')) as file:
            return [line.strip() for line in file ]
    except FileNotFoundError:
        print(f"Input file for day {day} not found")
        

def peek(dataset: Dataset, size: int = 5) -> Union[str, Dataset]:
    if len(dataset) > 1:
        return dataset[:size]
    if len(dataset) == 1:
        return dataset[0][:size]
    else:
        return []

def count(elements: list, predicate=bool) -> int:
    return sum(1 for each in elements if predicate(each))

def any_of(elements: list, predicate=bool) -> bool:
    return next((True for elements in elements if predicate(elements)), False)

def all_of(elements: list, predicate=bool) -> bool:
    return not any_of(elements, lambda x: not predicate(x))

log_level = 0
def log(message: str, level: int):
    if level <= log_level:
        print(message)

## Day 1

In [4]:
# input parsing

day1 = input_for(1)[0]
peek(day1)

'()()('

In [5]:
# part 1

def find_floor(input_values: Dataset) -> int:
    floor = 0
    for each in input_values:
        if each == '(':
            floor += 1
        elif each == ')':
            floor -= 1
        else:
            print('unknown character ' + each)
    return floor

find_floor(day1)

280

In [6]:
# part 2

def find_basement(input_values: Dataset) -> int:
    floor = 0
    for index, each in enumerate(input_values):
        if each == '(': floor += 1
        elif each == ')': floor -= 1
        else: print('unknown character ' + each)
        if floor == -1:
            return index + 1

find_basement(day1)

1797

## Day 2

In [7]:
# input parsing

def parse_line(line: str) -> (int, int, int):
    return [int(each) for each in line.split('x')]

day2 = [parse_line(line) for line in input_for(2)]
peek(day2)

[[20, 3, 11], [15, 27, 5], [6, 29, 7], [30, 15, 9], [19, 29, 21]]

In [8]:
# part 1

def needed_wrap_for(dimensions: (int, int, int)) -> int:
    areas = [first * second for (first, second) in itertools.combinations(dimensions, 2)]
    return min(areas) + 2 * sum(areas)

def needed_wrap_for_all(input_dataset: list[(int, int, int)]):
    return sum(map(needed_wrap_for, input_dataset))

needed_wrap_for_all(day2)

1606483

In [9]:
# part 2

def ribbon_length(sizes: (int, int, int)) -> int:
    return sum(sorted(sizes)[:2]) * 2

def bow_length(sizes: (int, int, int)) -> int:
    return sizes[0] * sizes[1] * sizes[2]

def ribbon_for_package(sizes: (int, int, int)) -> int:
    return ribbon_length(sizes) + bow_length(sizes)

def ribbon_for_all_packages(sizes: list[(int, int, int)]) -> int:
    return sum(map(ribbon_for_package, sizes))

ribbon_for_all_packages(day2)

3842356

## Day 3

In [10]:
# input parsing

day3 = input_for(3)[0]
peek(day3)

'>^^v^'

In [11]:
# part 1

directions = {
    '<': (-1, 0),
    '>': (1, 0),
    '^': (0, 1),
    'v': (0, -1)
}

# def visit_house(instruction):
#
#         if instruction == '<':
#             x -= 1
#         elif instruction == '>':
#             x += 1
#         elif instruction == '^':
#             y += 1
#         elif instruction == 'v':
#             y -= 1
#         else:
#             print(f'unmatched character {instruction}')

def visited_houses(instructions):
    
    position = (0, 0)
    
    visited = set()  # starting house
    visited.add(position)

    for instruction in instructions:
        dx, dy = directions[instruction]
        position = (position[0] + dx, position[1] + dy)
        
        visited.add(position)
    
    return visited

len(visited_houses(day3))

2592

In [12]:
# part 2

def parallel_visit(dataset: Dataset) -> set:
    return visited_houses(dataset[0::2]).union(visited_houses(dataset[1::2]))

len(parallel_visit(day3))

2360

## Day 4

In [13]:
day4 = 'ckczppom'

In [14]:
# part 1

from hashlib import md5

def hash_miner(key: str, zeros=5) -> int:

    bkey = key.encode()
    
    def hash_for(number: int) -> str:
        return md5(bkey + str(number).encode()).hexdigest()

    match = ''.join(['0' for _ in range(zeros)])
    
    for current in itertools.count(0):
        if hash_for(current)[:zeros] == match:
            return current

            
hash_miner(day4)

117946

In [15]:
# part 2

hash_miner(day4, zeros=6)

3938038

## Day 5

In [16]:
# input parsing

day5 = input_for(5)
peek(day5)

['zgsnvdmlfuplrubt',
 'vlhagaovgqjmgvwq',
 'ffumlmqwfcsyqpss',
 'zztdcqzqddaazdjp',
 'eavfzjajkjesnlsb']

In [17]:
# part 1

def day5_1(dataset: Dataset):
    rules_1 = (  # all the rules must be satisfied
        lambda s: len(list(filter(lambda c: c in 'aeiou', s))) >= 3,
        lambda s: re.compile(r'(.)\1').search(s) is not None,
        lambda s: next(filter(lambda x: x, (a == b for a, b in list(zip(s,s[1:])))), False),

        lambda s: all_of(['ab', 'cd', 'pq', 'xy'], lambda x: x not in s)
    )

    def validate_string_1(input_value: str) -> bool:
        return all_of(rules_1, lambda p: p(input_value))

    return count(dataset, validate_string_1)

day5_1(day5)

238

In [18]:
# part 2

def day5_2(dataset: Dataset):
    rules_2 = (
        lambda s: re.compile(r'(.)(.).*\1\2').search(s) is not None,
        lambda s: re.compile(r'(.).\1').search(s) is not None
    )

    def validate_string_2(input_value: str) -> bool:
        return all_of(rules_2, lambda p: p(input_value))

    def validate_all(dataset: Dataset) -> int:
        return count(dataset, validate_string_2)

    return validate_all(dataset)

day5_2(day5)

69

## Day 6

In [19]:
# input parsing

day6 = input_for(6)
peek(day6)

['turn on 489,959 through 759,964',
 'turn off 820,516 through 871,914',
 'turn off 427,423 through 929,502',
 'turn on 774,14 through 977,877',
 'turn on 410,146 through 864,337']

In [20]:
# part 1

# log_level = 0
Point = tuple[int, int]

def day6_1(instructions: Dataset) -> int:
    
    lights = {}
    
    def switch(action: str, start: Point, end: Point):
        log(f'action {action} from {start} through {end}', 1)
        for x in range(int(start[0]), int(end[0]) + 1):
            for y in range(int(start[1]), int(end[1]) + 1):
                if action == 'on':
                    turn_on(x, y)
                elif action == 'off':
                    turn_off(x, y)
                elif action == 'toggle':
                    toggle(x, y)

    def turn_on(x: int, y: int):
        lights[x, y] = True

    def turn_off(x: int, y: int):
        if (x, y) in lights:
            del lights[x, y]
    
    def toggle(x: int, y: int):
        if (x, y) in lights:
            turn_off(x, y)
        else:
            turn_on(x, y)
    
    
    def parse_instruction(instruction: str):
        tokens = instruction.split()
        if tokens[0] == 'toggle': switch('toggle', tokens[1].split(','), tokens[3].split(','))
        elif tokens[1] == 'on': switch('on', tokens[2].split(','), tokens[4].split(','))
        elif tokens[1] == 'off': switch('off', tokens[2].split(','), tokens[4].split(','))
        else: print(f'error parsing command: {instruction}')
#         print(tokens)
    
    for instruction in instructions:
        parse_instruction(instruction)
        log(f'{len(lights)} lights are on', 1)
        
    return len(lights)

day6_1(day6)
# list(map(lambda line: line.split(' ')[:2], day6))

569999

In [21]:
# part 2

def day6_2(instructions: Dataset) -> int:
    
    lights = {}
    
    def switch(action: str, start: Point, end: Point):
        log(f'action {action} from {start} through {end}', 1)
        for x in range(int(start[0]), int(end[0]) + 1):
            for y in range(int(start[1]), int(end[1]) + 1):
                if action == 'on':
                    turn_on(x, y)
                elif action == 'off':
                    turn_off(x, y)
                elif action == 'toggle':
                    toggle(x, y)

    def turn_on(x: int, y: int):
        lights[x, y] = lights.get((x, y), 0) + 1

    def turn_off(x: int, y: int):
        lights[x, y] = max(0, lights.get((x, y), 0) - 1)

    def toggle(x: int, y: int):
        lights[x, y] = lights.get((x, y), 0) + 2
    
    
    def parse_instruction(instruction: str):
        tokens = instruction.split()
        if tokens[0] == 'toggle': switch('toggle', tokens[1].split(','), tokens[3].split(','))
        elif tokens[1] == 'on': switch('on', tokens[2].split(','), tokens[4].split(','))
        elif tokens[1] == 'off': switch('off', tokens[2].split(','), tokens[4].split(','))
        else: print(f'error parsing command: {instruction}')
    
    for instruction in instructions:
        parse_instruction(instruction)
        log(f'{len(lights)} lights are on', 1)
        
    return sum(lights.values())

day6_2(day6)

17836115

## Day 7

In [22]:
# input parsing

day7 = input_for(7)
peek(day7)

['lf AND lq -> ls',
 'iu RSHIFT 1 -> jn',
 'bo OR bu -> bv',
 'gj RSHIFT 1 -> hc',
 'et RSHIFT 2 -> eu']

In [23]:
# part 1

def day7_1(dataset: Dataset, target='a', rewrite=False):

    graph = {}
    memo = {}

    def var(name: str) -> callable:
        if name not in memo:
            memo[name] = graph[name]()

        return memo[name]

    def expr(expr: str) -> callable:
        if expr.isdigit():
            return lambda: int(expr)
        else:
            return lambda: var(expr)

    def build_graph(instructions: Dataset):
        for instruction in instructions:
            parse_line(instruction)

    def parse_line(instruction: str):
        left, right = instruction.split(' -> ')
        if not rewrite:
            assert right not in graph  # only one definition for variable
        graph[right] = parse_instruction(left)

    def parse_instruction(instruction: str) -> callable:
        tokens = instruction.split()
        if len(tokens) == 1:  # 123 -> x
            return expr(tokens[0])

        if len(tokens) == 2 and tokens[0] == 'NOT':  # NOT e -> f
            return lambda: 65_535 ^ var(tokens[1])  # negate all the 16 bits

        elif len(tokens) == 3:
            left = expr(tokens[0])
            operator = tokens[1]
            right = expr(tokens[2])
            if operator == 'AND':  # x AND y -> z
                return lambda: left() & right()
            elif operator == 'OR':  # x OR y -> z
                return lambda: left() | right()
            elif operator == 'LSHIFT':  # x OR y -> z
                return lambda: left() << right()
            elif operator == 'RSHIFT':  # x OR y -> z
                return lambda: left() >> right()

        else:
            print(f'error parsing instruction: {instruction}')

    build_graph(dataset)
    result = graph[target]()
    log(memo, level=1)

    return result


day7_1(day7)

16076

In [24]:
# part 2

def day7_2(dataset: Dataset, target: str = 'a') -> int:
    return day7_1(dataset, target=target, rewrite=True)

day7_2(day7 + ['16076 -> b'])

2797

## Day 8

In [25]:
# input parsing

day8 = input_for(8)
peek(day8)

['"qxfcsmh"',
 '"ffsfyxbyuhqkpwatkjgudo"',
 '"byc\\x9dyxuafof\\\\\\xa6uf\\\\axfozomj\\\\olh\\x6a"',
 '"jtqvz"',
 '"uzezxa\\"jgbmojtwyfbfguz"']

In [26]:
# part 1
def memory(line: str, replace_rules):
    return functools.reduce(lambda s, rule: re.sub(rule[0], rule[1], s),
                                replace_rules.items(), line)


def day8_1(dataset: Dataset):

    replace_rules = {  # dictionaries are now sorted in python
        r'\\x[0-9a-f]{2}': 'U',
        r'\\[\\"]': 'E',
        r'"': ''
    }

    def diff(line: str) -> int:
        encoded = memory(line, replace_rules)
        # print(f'{line} -> {encoded} — {len(line)}, {len(encoded)}')
        return len(line) - len(encoded)

    return sum(map(diff, dataset))


day8_1(day8)

1350

In [27]:
def day8_2(dataset: Dataset) -> int:

    replace_rules = {
        r'\\x[0-9a-f]{2}': r'UUxUU',
        r'\\[\\"]': 'EEEE',
        r'"': r'\"'
    }

    def diff(line: str) -> int:
        encoded = "\"" + memory(line, replace_rules) + "\""
        # print(f'{line} -> {encoded} — {len(line)}, {len(encoded)}')
        return len(encoded) - len(line)

    return sum(map(diff, dataset))


day8_2(day8)

2085

## Day 9

In [28]:
# input parsing

day9 = input_for(9)
peek(day9)

['Faerun to Tristram = 65',
 'Faerun to Tambi = 129',
 'Faerun to Norrath = 144',
 'Faerun to Snowdin = 71',
 'Faerun to Straylight = 137']

In [29]:
# part 1

def day9_1(dataset: Dataset, minimize=True):
    
    line_matcher = re.compile(r'(\w+)\s+to\s+(\w+)\s+=\s+(\d+)')
    distances = {}
    
    def parse_line(line: str) -> tuple[str, str, int]:
        return line_matcher.match(line).groups()
    
    def generate_distances_map(distances: list[tuple[str, str, int]]):
        for first, second, length in distances:
            add_distance(first, second, int(length))
    
    def add_distance(first: str, second: str, length: int):
        if first not in distances:
            distances[first] = {}
        if second not in distances:
            distances[second] = {}
        distances[first][second] = length
        distances[second][first] = length
    
    def route_distance(path: tuple) -> int:
        return sum(distances[first][second] for first, second in zip(path, path[1:]))
        
    generate_distances_map(map(parse_line, dataset))

    route_generator = map(route_distance, itertools.permutations(distances.keys()))
    if minimize:
        return min(route_generator)
    else:
        return max(route_generator)

day9_1(day9)

117

In [30]:
# part 2

def day9_2(dataset):
    return day9_1(dataset, minimize=False)

day9_2(day9)

909

## Day 10

In [31]:
# input parsing
day10 = '1113222113'
day10

'1113222113'

In [32]:
# part 1

def day10_1(seed: int, iterations=40):
    
    def encode(number: str) -> str:
        count = 1
        found = []
        for previous, current in zip(number, number[1:]):
            if previous != current:
                found.append((str(count), previous))
                count = 0
            count += 1
        found.append((str(count), number[-1]))
        return ''.join(itertools.chain.from_iterable(found))

    result = seed
    for _ in range(iterations):
        result = encode(result)
    return len(result)

day10_1(day10, 40)

252594

In [33]:
# part 2

def day10_2(seed: int):
    return day10_1(seed, 50)

day10_2(day10)

3579328